In [3]:
!pip install requests beautifulsoup4 pandas tqdm --quiet


In [4]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm


In [5]:
BASE_URL = "https://www.sd.go.kr/main/webRecoveryCouponList.do"
HEADERS  = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
KEY      = "5269"  # 성동구 key

def scrape_page(page_idx):
    params = {
        "searchName":     "",      # 상호
        "searchEmdNm":    "",      # 법정동
        "searchAddress":  "",      # 주소
        "searchBizRegNo": "",      # 사업자번호
        "key":            KEY,
        "pageIndex":      page_idx
    }
    resp = requests.get(BASE_URL, headers=HEADERS, params=params)
    resp.raise_for_status()

    soup = BeautifulSoup(resp.text, "html.parser")
    out = []

    for tr in soup.select("table.responsive tbody tr"):
        # <th>에 데이터가 들어 있으니 td, th 둘 다 가져옵니다
        cells = tr.find_all(["td","th"])
        if len(cells) >= 3:
            out.append({
                "상호":   cells[0].get_text(strip=True),
                "소재지": cells[1].get_text(strip=True),
                "주소":   cells[2].get_text(strip=True)
            })
    return out

In [6]:
sample = scrape_page(1)
print("1페이지 건수:", len(sample))
sample[:5]


1페이지 건수: 10


[{'상호': '거성레이져', '소재지': '성수동1가', '주소': '서울 성동구 상원길 23(성수동1가)'},
 {'상호': '명작 김진향한복', '소재지': '금호동4가', '주소': '서울 성동구 독서당로294, 2층 (금호동4가)'},
 {'상호': '남도집', '소재지': '도선동', '주소': '서울 성동구 왕십리로20길 9-1 (도선동)'},
 {'상호': '신도데이타', '소재지': '성수동2가', '주소': '서울 성동구 성수이로10길 81층(성수동2가)'},
 {'상호': '짱텔레콤', '소재지': '용답동', '주소': '서울 성동구 용답중앙15길 31층(용답동)'}]

In [7]:
records = []
for pg in tqdm(range(1, 1682), desc="페이지 스크래핑"):
    data = scrape_page(pg)
    if not data:
        break
    records.extend(data)
    time.sleep(0.2)

df = pd.DataFrame(records)
df.head(10)

페이지 스크래핑:   0%|          | 0/1681 [00:00<?, ?it/s]

,상호,소재지,주소
0,거성레이져,성수동1가,서울 성동구 상원길 23(성수동1가)
1,명작 김진향한복,금호동4가,"서울 성동구 독서당로294, 2층 (금호동4가)"
2,남도집,도선동,서울 성동구 왕십리로20길 9-1 (도선동)
3,신도데이타,성수동2가,서울 성동구 성수이로10길 81층(성수동2가)
4,짱텔레콤,용답동,서울 성동구 용답중앙15길 31층(용답동)
5,법무사이두석사무소,성수동1가,"서울 성동구 상원6나길 22-11, 205호 (성수동1가, 성동지역경제혁신센터)"
6,김종용누룽지통닭,행당동,"서울 성동구 마조로5길 7 1,2층"
7,깜장커피,행당동,서울 성동구 왕십리로 223 2층
8,씨유마장현대점,마장동,서울 성동구 마장동 768-1 안민빌딩 109
9,홍단,성수동1가,"서울특별시 성동구 광나루로 130 1206호 (성수동1가,12층)"


In [8]:
df.to_csv("seongdong_coupons.csv", index=False, encoding="utf-8-sig")
print(f"✅ 총 {len(df)}건 저장 완료 → seongdong_coupons.csv")


✅ 총 16802건 저장 완료 → seongdong_coupons.csv
